In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, BatchNormalization, Dense
from tensorflow.keras.utils import to_categorical
import tkinter as tk
from PIL import Image, ImageDraw, ImageOps


# ------------------------------
# Load and preprocess MNIST data
# ------------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize images
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Reshape for CNN input
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

# One-hot encode labels
num_classes = 10
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(f"Processed x_train shape: {x_train.shape}")


# ------------------------------
# Define CNN model
# ------------------------------
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, concatenate, Dropout, BatchNormalization, Dense

def create_advanced_mnist_model(input_shape=(28,28,1), num_classes=10):
    inputs = Input(shape=input_shape)
    
    # Pathway 1: Standard convolution
    x1 = Conv2D(32, (3,3), activation='relu', padding='same')(inputs)
    x1 = Conv2D(32, (3,3), activation='relu', padding='same')(x1)
    x1 = MaxPooling2D((2,2))(x1)
    x1 = Dropout(0.25)(x1)
    
    # Pathway 2: Larger receptive field
    x2 = Conv2D(32, (5,5), activation='relu', padding='same')(inputs)
    x2 = Conv2D(32, (5,5), activation='relu', padding='same')(x2)
    x2 = MaxPooling2D((2,2))(x2)
    x2 = Dropout(0.25)(x2)
    
    # Merge pathways
    merged = concatenate([x1, x2])
    
    # Continue processing
    x = Conv2D(64, (3,3), activation='relu', padding='same')(merged)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2,2))(x)
    x = Dropout(0.25)(x)
    
    x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.25)(x)
    
    # Global Average Pooling instead of Flatten
    x = layers.GlobalAveragePooling2D()(x)
    
    # Dense layers
    x = Dense(256, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    outputs = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs)
    return model



Processed x_train shape: (60000, 28, 28, 1)


In [2]:
try:
    model = tf.keras.models.load_model('mnist_cnn_model.keras')
    print("Loaded saved model.")
except:
    print("Training new model...")
    model = create_advanced_mnist_model()
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    history = model.fit(x_train, y_train, epochs=30, validation_data=(x_test,y_test), batch_size=64)
    model.save('mnist_cnn_model.keras')

test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy:.4f}")



Loaded saved model.
313/313 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.9942 - loss: 0.0214
Test accuracy: 0.9950


In [3]:
y_pred = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step


In [6]:
y_pred_label = np.argmax(y_pred, axis=1)

In [9]:
y_true_label = np.argmax(y_pred, axis=1)

In [10]:
from sklearn.metrics import confusion_matrix

Confusion matrix

In [13]:
print(confusion_matrix(y_true=y_true_label, y_pred= y_pred_label))

[[ 983    0    0    0    0    0    0    0    0    0]
 [   0 1137    0    0    0    0    0    0    0    0]
 [   0    0 1029    0    0    0    0    0    0    0]
 [   0    0    0 1021    0    0    0    0    0    0]
 [   0    0    0    0  979    0    0    0    0    0]
 [   0    0    0    0    0  887    0    0    0    0]
 [   0    0    0    0    0    0  951    0    0    0]
 [   0    0    0    0    0    0    0 1027    0    0]
 [   0    0    0    0    0    0    0    0  975    0]
 [   0    0    0    0    0    0    0    0    0 1011]]
